In [1]:
import os
from os import path
import pandas as pd

In [8]:
# PATHS AND CONSTANTS

OUR_DATA_PATH = "/gpfs/slayman/pi/gerstein/cjy23/project/PsycENCODE/analysis/network/results/combined" 
# /results/original for data separated by chromosone

EPIMAP_DATA_PATH = "/gpfs/slayman/pi/gerstein/rah86/project/PsycENCODE/data/epimap_bygroup_bed"
EPIMAP_DATA_SORTED_PATH = "/gpfs/slayman/pi/gerstein/rah86/project/PsycENCODE/data/sorted_epimap_bygroup_bed"

## OBJECTIVES
1. Count # of unique genes in epimap data
2. Calculate overlap between networks of different tissues in the epimap data
3. What does "brain" mean in the context of this epimap data? Look at the paper

Can use os.system()

## 1: Unique genes in epimap data
* By Tissue
* Total

In [20]:
gene_set = set()
gene_counts = {}
for file in os.listdir(EPIMAP_DATA_PATH):
    if file[-3:] == 'tsv':
        a = set(pd.read_table(path.join(EPIMAP_DATA_PATH,file),delimiter='\t' )['gene'].tolist())
        gene_set = gene_set.union(a)
        gene_counts[file[:-4]] = len(a)

In [21]:
len(gene_set)

19028

In [22]:
with open('all_genes.txt', 'w') as ff:
    ff.write("\n".join(list(gene_set)))
with open('n_genes_by_region.txt','w') as ff:
    ff.write("total: "+ str(len(gene_set))+'\n')
    for key, l in gene_counts.items():
        ff.write(key + ": " + str(l)+"\n")

## 2. What does "Brain" Mean in the epimap data?
* Brain is an amalgamation of the measurement of many brain regions accross gender, cell type, etc
* Data is available by going [here](https://cboix.shinyapps.io/epimap_vis/) and clicking on the `Brain` folder on the left

## 3. Overlap between the networks of different tissues in epimap data

In [26]:
import plotly.express as px
import numpy as np

In [29]:
files = os.listdir(EPIMAP_DATA_PATH)
files.remove('all_ccres_epimap.bed') # not applicable

matrix = np.eye(N=len(files)) * 0.5 # eye because overlap with self is always 1. ... 0.5 because I will add to self
#name_to_ix = dict(zip(files,list(range(len(files)))))
for i, file1 in enumerate(files):
    for i2 in range(i+1, len(files)):
        file2 = files[i2]
        #print(file1, ' ', file2)
        # get jaccard info, save to tmp file
        os.system('bedtools jaccard -a {file1} -b {file2}'.format(file1=os.path.join(EPIMAP_DATA_SORTED_PATH,file1) , file2=os.path.join(EPIMAP_DATA_SORTED_PATH,file2)) + ' > tmp')
        # read result out of tmp file
        # other info available: intersection	union-intersection	jaccard	n_intersections
        try:
            jaccard = pd.read_table('tmp',delimiter='\t' )['jaccard'].item()
        except:
            print('ERROR ', file1, file2)
            jaccard = 0
        matrix[i,i2] = jaccard # calculation here

df = pd.DataFrame(matrix, index = files, columns = files)
df = df+df.T # reflect over diagonal

ERROR  epimap_pancreas.bed all_ccres_epimap.bed
ERROR  epimap_es_deriv.bed all_ccres_epimap.bed
ERROR  epimap_mesench.bed all_ccres_epimap.bed
ERROR  epimap_pns.bed all_ccres_epimap.bed
ERROR  epimap_adipose.bed all_ccres_epimap.bed
ERROR  epimap_endocrine.bed all_ccres_epimap.bed
ERROR  epimap_digestive.bed all_ccres_epimap.bed
ERROR  all_ccres_epimap.bed epimap_ipsc.bed
ERROR  all_ccres_epimap.bed epimap_lymphoblastoid.bed
ERROR  all_ccres_epimap.bed epimap_hsc_b_cell.bed
ERROR  all_ccres_epimap.bed epimap_muscle.bed
ERROR  all_ccres_epimap.bed epimap_myosat.bed
ERROR  all_ccres_epimap.bed epimap_stromal.bed
ERROR  all_ccres_epimap.bed epimap_bone.bed
ERROR  all_ccres_epimap.bed epimap_urinary.bed
ERROR  all_ccres_epimap.bed epimap_lung.bed
ERROR  all_ccres_epimap.bed epimap_neurosph.bed
ERROR  all_ccres_epimap.bed epimap_kidney.bed
ERROR  all_ccres_epimap.bed epimap_placenta_eem.bed
ERROR  all_ccres_epimap.bed epimap_reproductive.bed
ERROR  all_ccres_epimap.bed epimap_heart.bed
ERRO

In [31]:
px.imshow(df).write_html('./similarity_matrix.html')

In [20]:
os.system('bedtools jaccard -a {file1} -b {file2}'.format(file1=os.path.join(EPIMAP_DATA_SORTED_PATH,file1) , file2=os.path.join(EPIMAP_DATA_SORTED_PATH,file2)) + ' > tmp')

0

In [15]:
z='bedtools jaccard -a {file1} -b {file2}'.format(file1=os.path.join(EPIMAP_DATA_SORTED_PATH,file1) , file2=os.path.join(EPIMAP_DATA_SORTED_PATH,file2))

In [16]:
z

'bedtools jaccard -a /gpfs/slayman/pi/gerstein/rah86/project/PsycENCODE/data/sorted_epimap_bygroup_bed/epimap_endothelial.bed -b /gpfs/slayman/pi/gerstein/rah86/project/PsycENCODE/data/sorted_epimap_bygroup_bed/epimap_endothelial.bed'

In [14]:
os.path.join(EPIMAP_DATA_SORTED_PATH,file1)

'/gpfs/slayman/pi/gerstein/rah86/project/PsycENCODE/data/sorted_epimap_bygroup_bed/epimap_endothelial.bed'

In [25]:
pd.read_table('tmp',delimiter='\t' )

,intersection,union-intersection,jaccard,n_intersections
0,53922869,53922869,1,189738
